### Polymorphism:
- The ability to define a generic type of behavior that will (potentially) behave differently when applied to different types. 

<br>

### Special Methods
- class instantiation
    * \__init__
- context managers
    * \__enter__
    * \__exit__
- sequence types
    * \__getitem__
    * \__setitem__
    * \__delitem__
- iterables and iterators
    * \__iter__
    * \__next__
    * \__len__
- in operator
    * \__contains__

<br>

### \__str\__ vs \__repr__

In [1]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def __str__(self):
        return self.name
    
    def __repr__(self):
        return f"Person(name='{self.name}', age='{self.age}')"

In [2]:
p = Person(name='Theo', age=2)
print(repr(p))
print(p)

Person(name='Theo', age='2')
Theo


### Arithmetic operators

- \__add__
- \__sub__
- \__mul__
- \__truediv__
- \__floordiv__
- \__mod__
- \__pow__

<br>
to indicate the operation is not supported, implement method and return NotImplemented

In [25]:
from numbers import Real
import math

class VectorDimensionMismatch(TypeError):
    pass

class Vector:
    """A one-dimensional array data structure"""
    def __init__(self, *components):
        if len(components) < 1:
            raise ValueError('Cannot create an empty vector.')
        for component in components:
            if not isinstance(component, Real):
                raise ValueError('Vector components must all be real numbers.')
        self._components = tuple(components)
    
    def __len__(self):
        return len(self._components)
    
    @property
    def components(self):
        return self._components
    
    def __repr__(self):
        return f'Vector{self.components}'

    def validate_type_and_dimension(self, v):
        return isinstance(v, Vector) and len(v) == len(self)
    
    def __add__(self, other):
        if not self.validate_type_and_dimension(other):
            return NotImplemented
        components = (x + y for x, y in zip(self.components, other.components))
        return Vector(*components)
    
    def __sub__(self, other):
        if not self.validate_type_and_dimension(other):
             return NotImplemented
        components = (x - y for x, y in zip(self.components, other.components))
        return Vector(*components)
    
    def __mul__(self, other):
        if isinstance(other, Real):
            components = (other * x for x in self.components)
            return Vector(*components)
        if self.validate_type_and_dimension(other):
            # dot product
            components = (x * y for x, y in zip(self.components, other.components))
            return sum(components)
        return NotImplemented

    def __rmul__(self, other):
        return self * other
    
    def __iadd__(self, other):
        if self.validate_type_and_dimension(other):
            components = (x + y for x, y in zip(self.components, other.components))
            self._components = tuple(components)
            return self     
        return NotImplemented
    
    def __neg__(self):
        components = (-x for x in self.components)
        return Vector(*components)
    
    def __abs__(self):
        return math.sqrt(sum(x**2 for x in self.components))
    
    # Rich comparisons
    def __eq__(self, other):
        if isinstance(other, tuple):
            other = Vector(*other)
        if isinstance(other, Vector):
            return self.x == other.x and self.y == other.y
        return NotImplemented
    
    def __lt__(self, value):
        if isinstance(other, tuple):
            other = Vector(*other)
        if isinstance(other, Vector):
            return abs(self) < abs(other)

    def __le__(self, value):
        return self == other or self < other
    
    def __hash__(self):
        return hash(self.components)

    def __bool__(self):
        return len(self) > 0




In [26]:
v1 = Vector(1, 2)
v2 = Vector(10, 20)
v3 = Vector(100, 200, 300)
v4 = Vector(-5, -10)

In [14]:
print('add: ', v1 + v2)
print('add - : ', v1 + -v2)
print('multiply: ', v1 * v2)
print('multiply: ', 3 * v1)
print('abs: ', abs(v4))

add:  Vector(11, 22)
add - :  Vector(-9, -18)
multiply:  50
multiply:  Vector(3, 6)
abs:  11.180339887498949


In [27]:
bool(v1)

Vector(1, 2)


True

### Callables

In [6]:
class Partial:

    def __init__(self, func, *args):
        self._func = func
        self._args = args
    
    def __call__(self, *args):
        return self._func(*self._args, *args)
        

In [29]:
from collections import defaultdict

class DefaultValue:

    def __init__(self, default_value):
        self.default_value = default_value
        self.counter = 0
    
    
    def __call__(self):
        self.counter += 1
        return self.default_value


cache_def = DefaultValue('N/A')
cache = defaultdict(cache_def)

cache['a'] = 100
print(cache['b'])
print(cache.items())
print(cache_def.counter)

N/A
dict_items([('a', 100), ('b', 'N/A')])
1


## class-based decorator

In [32]:
from time import perf_counter
from functools import wraps
from time import sleep
import random

class Profiler:

    def __init__(self, fn):
        self.counter = 0
        self.total_elapsed = 0
        self.fn = fn
    
    def __call__(self, *args, **kwargs):
        self.counter += 1
        start = perf_counter()
        result = self.fn(*args, **kwargs)
        end = perf_counter()
        self.total_elapsed += (end - start)
        return result
    
    @property
    def avg_time(self):
        return self.total_elapsed / self.counter


In [35]:
@Profiler
def func_1(a, b):
    sleep(random.random())
    return (a, b)

print(func_1(10, 20))
print(func_1.counter)
print(func_1.avg_time)

(10, 20)
1
0.9101682000036817
